<header style="width:100%;position:relative">
  <div style="width:80%;float:right;">
    <h1>False Political Claim Detection</h1>
    <h3>Carga y limpieza de los datos - LLM</h3>
    <h5>Grupo 2</h5>
  </div>
        <img style="width:15%;" src="./images/logo.jpg" alt="UPM" />
</header>

# Índice

1. [Importar librerias](#1.-Importar-librerias)  
2. [Funciones auxiliares](#2.-Funciones-auxiliares)  
3. [Configuracion de los diccionarios](#3.-configuracion-de-los-diccionarios)  
4. [Creacion de One-Hot](#4.-Creacion-de-One-Hot)  
5. [Exportar CSV](#5.-Exportar-CSV)  
6. [Referencias](#6-referencias)

# 1. Importar Librerias

In [ ]:
import pandas as pd
import numpy as np

from datetime import datetime
import json


import matplotlib.pyplot as plt
import seaborn as sns


import spacy

import re
from multiprocessing import process
from transformers import pipeline


from collections import Counter


# pip install gensim
import gensim
from gensim.corpora import Dictionary

# pip install pyLDAvis==3.4.1
# import pyLDAvis

# 2. Carga del Dataset

In [ ]:
# Train
url = "formated/train_exportado.csv" 
df_train_exportado = pd.read_csv(url)


# Test
url2 = "formated/test_exportado.csv"
df_test_exportado = pd.read_csv(url2)

In [ ]:
df_train_exportado_words_onehot = df_train_exportado[['id', 'label']]
df_test_exportado_words_onehot = df_test_exportado[['id']]

# 3. Configuracion de los diccionarios

In [ ]:
statement = 'statement-lemmatize'

In [ ]:
nlp = spacy.load("en_core_web_lg")  # Carga el modelo de spaCy

def clean_statements(statements):
    global nlp
    """
    Elimina pronombres, palabras vacías, palabras irrelevantes y tokens de longitud 1 de las declaraciones.

    Args:
        statements (list of str): Lista de declaraciones a limpiar.
        nlp (spacy.Language): Modelo de spaCy cargado.

    Returns:
        list of str: Lista de declaraciones limpias.
    """
    # Combina las stop words de spaCy con las personalizadas
    custom_stopwords = {"i", "we", "it", "he", "she", "they", "our", "that", "have", "be", "say"}
    all_stopwords = nlp.Defaults.stop_words.union(custom_stopwords)

    cleaned_statements = []
    for statement in statements:
        doc = nlp(statement)
        cleaned_tokens = []
        for token in doc:
            # Excluir palabras que son stop words, están en la lista personalizada o tienen longitud 1
            if token.text.lower() in all_stopwords or token.is_stop or len(token.text) == 1 or len(token.text) == 2:
                continue
            # Incluir solo palabras relevantes
            if token.pos_ in {"VERB", "NOUN", "PROPN", "ADJ"}:
                cleaned_tokens.append(token.text.lower())
        cleaned_statements.append(" ".join(cleaned_tokens))
    return cleaned_statements

In [ ]:
print(nlp.Defaults.stop_words)

In [ ]:
df_train_exportado[statement] = clean_statements(df_train_exportado[statement]) 

In [ ]:
df_test_exportado[statement] = clean_statements(df_test_exportado[statement]) 


In [ ]:
print(df_train_exportado[statement].head(5))

In [ ]:
def generar_array_serie(inicio, fin):
  """
  Genera un array que sigue la serie 4, 7, 11, 16, 22... hasta alcanzar o superar el valor final.

  Args:
    inicio: El primer número de la serie.
    fin: El valor máximo que el último elemento del array no debe superar.

  Returns:
    Una lista (array) con los números de la serie.
  """
  array = [inicio]
  ultimo_elemento = inicio
#   incremento = 3
  incremento = 1
  while ultimo_elemento < fin:
    ultimo_elemento += incremento
    array.append(ultimo_elemento)
    incremento += 1
  return array

mi_array = generar_array_serie(4, 270)
# mi_array = generar_array_serie(2, 270)
print(mi_array)

In [ ]:
# Lista de valores mínimos de frecuencia. Una palabra debe aparecer al menos este número de veces en los documentos para ser incluida en el diccionario.
# min_freq_values = [5, 10, 20, 50]
# min_freq_values = [10, 20, 30, 40 ,50]
# min_freq_values = [5,10 ,20, 40, 60 ,80,100]
# min_freq_values = [4,7,10 ,20, 40, 60 ,80,100]
# min_freq_values = [4,7,10 ,20, 35, 55 ,80,105, 135]
min_freq_values =   mi_array   


# Lista de proporciones máximas. Una palabra no debe aparecer en más del porcentaje especificado de documentos para ser incluida en el diccionario.
# max_prop_values = [0.5, 0.65, 0.8]
max_prop_values = [0.4]


In [ ]:
# Separar en dos listas según la etiqueta
statements_true = [str(text).split() for text, label in zip(df_train_exportado[statement], df_train_exportado['label']) if label == 1]
statements_false =[str(text).split() for text, label in zip(df_train_exportado[statement], df_train_exportado['label']) if label == 0]

print(len(statements_true))
print(len(statements_false))

In [ ]:
# Cuenta en cuántos documentos aparece cada palabra
def document_frequency(tokenized_docs):
    df_counter = Counter()
    for doc in tokenized_docs:
        unique_words = set(doc)
        df_counter.update(unique_words)
    return df_counter

# Función para filtrar palabras por frecuencia mínima y máxima proporción
def filtra(dic_frecuencia, doc_frecuencia, total_docs, min_freq, max_prop):
    return {
        word: dic_frecuencia[word]  # frecuencia total (recuento)
        for word in dic_frecuencia
        if doc_frecuencia[word] >= min_freq and (doc_frecuencia[word] / total_docs) <= max_prop
    }

# Tokenización según clase
statements_true = [str(text).split() for text, label in zip(df_train_exportado[statement], df_train_exportado['label']) if label == 1]
statements_false = [str(text).split() for text, label in zip(df_train_exportado[statement], df_train_exportado['label']) if label == 0]

# Estadísticas base
n_docs_true = len(statements_true)
n_docs_false = len(statements_false)
adjust_ratio = n_docs_true / n_docs_false

# Contadores
counter_true = Counter(word for doc in statements_true for word in doc)
counter_false = Counter(word for doc in statements_false for word in doc)
df_true = document_frequency(statements_true)
df_false = document_frequency(statements_false)

# Procesa combinaciones
for min_freq_true in min_freq_values:
    min_freq_false = max(1, round(min_freq_true / adjust_ratio))  # Ajuste proporcional

    for max_prop in max_prop_values:
        # Filtrado
        filtered_true = filtra(counter_true, df_true, n_docs_true, min_freq_true, max_prop)
        filtered_false = filtra(counter_false, df_false, n_docs_false, min_freq_false, max_prop)

        # Palabras en común
        common_words = set(filtered_true.keys()) & set(filtered_false.keys())

        # Exclusivos (frecuencia absoluta)
        exclusive_true = {
            word: freq for word, freq in filtered_true.items() if word not in common_words
        }
        exclusive_false = {
            word: freq for word, freq in filtered_false.items() if word not in common_words
        }

        # Opcional: ordenar por frecuencia descendente
        filtered_true = dict(sorted(filtered_true.items(), key=lambda x: x[1], reverse=True))
        filtered_false = dict(sorted(filtered_false.items(), key=lambda x: x[1], reverse=True))
        exclusive_true = dict(sorted(exclusive_true.items(), key=lambda x: x[1], reverse=True))
        exclusive_false = dict(sorted(exclusive_false.items(), key=lambda x: x[1], reverse=True))

        # Guarda resultados
        base = f"nb{min_freq_true}_na{int(max_prop * 100)}"
        with open(f"json/dictionaries/true_{base}.json", "w") as f:
            json.dump(filtered_true, f, indent=2)
        with open(f"json/dictionaries/false_{base}.json", "w") as f:
            json.dump(filtered_false, f, indent=2)
        with open(f"json/dictionaries/true_exclusive_{base}.json", "w") as f:
            json.dump(exclusive_true, f, indent=2)
        with open(f"json/dictionaries/false_exclusive_{base}.json", "w") as f:
            json.dump(exclusive_false, f, indent=2)

        print(f"✔ Guardado con min_freq_true={min_freq_true}, min_freq_false={min_freq_false}, max_prop={max_prop}")


In [ ]:
import json
from collections import defaultdict
from math import log
from math import sqrt
from math import exp

# 10, 20, 40, 60 ,80,100
# Cargar diccionarios
# nb = [10, 20, 40, 60 ,80,100]
nb = min_freq_values


diccionario_true = []
diccionario_false = []

# Logaritmos
# for n in nb:
#     diccionario_true.append((f"json/dictionaries/true_nb{n}_na40.json", 1/log(n)))
#     diccionario_false.append((f"json/dictionaries/false_nb{n}_na40.json", 1/log(n)))

def calculate_weights(nb):
    """
    Calcula los pesos para los diccionarios true y false basados en los valores de nb.

    Args:
        nb (list): Lista de valores mínimos de frecuencia.

    Returns:
        tuple: Dos listas de tuplas con rutas de diccionarios y sus pesos correspondientes.
    """
    diccionario_true = []
    diccionario_false = []

    for n in nb:
        # Logaritmos normalizados
        # weight = 1 - log(n) / log(max(nb))
        
        # Raiz de logaritmos
        # weight = 1 / sqrt(log(n))

        # Exponencial negativa:
        # weight = exp(-n / max(nb))
        
        # Inversa al cuadrado:
        # weight = 1 / (n ** 2)

        # Peso por rareza relativa: 67,9
        weight = 1 / (n - min(nb) + 1)

        # Escalado lineal invertido:
        # weight = (max(nb) - n) / (max(nb) - min(nb))

        # Escalado logístico:


        diccionario_true.append((f"json/dictionaries/true_nb{n}_na40.json", weight))
        diccionario_false.append((f"json/dictionaries/false_nb{n}_na40.json", weight))
    
    return diccionario_true, diccionario_false


# Llamar a la función
diccionario_true, diccionario_false = calculate_weights(nb)


def construir_diccionario_ponderado(lista_archivos_y_pesos):
    resultado = defaultdict(float)
    for ruta, peso in lista_archivos_y_pesos:
        with open(ruta, "r") as f:
            datos = json.load(f)
            for palabra in datos:
                resultado[palabra] += peso
    return dict(resultado)

# Crear diccionarios ponderados
diccionario_true_ponderado = construir_diccionario_ponderado(diccionario_true)
diccionario_false_ponderado = construir_diccionario_ponderado(diccionario_false)

# Opcional: guardar resultados
with open("json/dictionaries_ponderados/diccionario_true_ponderado.json", "w") as f:
    json.dump(diccionario_true_ponderado, f, indent=2)

with open("json/dictionaries_ponderados/diccionario_false_ponderado.json", "w") as f:
    json.dump(diccionario_false_ponderado, f, indent=2)


In [ ]:
import pandas as pd
import json
import re

# Cargar los diccionarios ponderados
with open("json/dictionaries_ponderados/diccionario_true_ponderado.json") as f:
    diccionario_true_ponderado = json.load(f)

with open("json/dictionaries_ponderados/diccionario_false_ponderado.json") as f:
    diccionario_false_ponderado = json.load(f)


# Función para calcular el valor de una statement
def calcular_score(statement):
    # Preprocesar: pasar a minúsculas, quitar puntuación
    palabras = re.findall(r"\b\w+\b", statement.lower())
    score = 0.0
    for palabra in palabras:
        score += diccionario_true_ponderado.get(palabra, 0)
        score -= diccionario_false_ponderado.get(palabra, 0)
    return score

# Aplicar la función al dataset
# df["statement_score"] = df["statement"].apply(calcular_score)
# df_train_exportado_words_onehot["statement_score"] = df_train_exportado_words_onehot[statement].apply(calcular_score)
# df_test_exportado_words_onehot["statement_score"] = df_test_exportado_words_onehot[statement].apply(calcular_score)
df_train_exportado_words_onehot["statement_score"] = df_train_exportado[statement].apply(calcular_score)
df_test_exportado_words_onehot["statement_score"] = df_test_exportado[statement].apply(calcular_score)

print(df_train_exportado_words_onehot.head(5))
# Guardar el dataset con la nueva columna (opcional)
# df.to_csv("data/train_con_score.csv", index=False)

# 99 Busqueda de mejor manera de evaluar


# 3.5

# 4. Puntuacíón por diccionarios 

# 4. Creacion de One-Hot

# 5. One Hot - variables categóricas 

## 5.0 Subject

In [ ]:
# print(df_train_exportado.columns)

# Filtrar las columnas que comienzan con 'subject_'
subject_columns = [col for col in df_train_exportado.columns if col.startswith('subject-')]
print(subject_columns)
# Agregar las columnas filtradas al DataFrame words_onehot
df_train_exportado_words_onehot = pd.concat([df_train_exportado_words_onehot, df_train_exportado[subject_columns]], axis=1)
df_test_exportado_words_onehot = pd.concat([df_test_exportado_words_onehot, df_test_exportado[subject_columns]], axis=1)

print("Columnas 'subject_' añadidas correctamente a los DataFrames words_onehot.")

## 5.1 speaker

In [ ]:
df_train_exportado_words_onehot['speaker_grouped'] = df_train_exportado['speaker_grouped']
df_test_exportado_words_onehot['speaker_grouped'] = df_test_exportado['speaker_grouped']

In [ ]:
df_train_exportado_words_onehot = pd.get_dummies(df_train_exportado_words_onehot, columns=['speaker_grouped'], prefix='speaker_grouped', dtype=int)
df_test_exportado_words_onehot = pd.get_dummies(df_test_exportado_words_onehot, columns=['speaker_grouped'], prefix='speaker_grouped', dtype=int)

## 5.2 speaker_job

In [ ]:
df_train_exportado_words_onehot['speaker_job_grouped'] = df_train_exportado['speaker_job_grouped']
df_test_exportado_words_onehot['speaker_job_grouped'] = df_test_exportado['speaker_job_grouped']

In [ ]:
df_train_exportado_words_onehot = pd.get_dummies(df_train_exportado_words_onehot, columns=['speaker_job_grouped'], prefix='speaker_job_grouped', dtype=int)
df_test_exportado_words_onehot = pd.get_dummies(df_test_exportado_words_onehot, columns=['speaker_job_grouped'], prefix='speaker_job_grouped', dtype=int)

## 5.3 state_info

In [ ]:
df_test_exportado_words_onehot['state_info_grouped'] = df_test_exportado['state_info_grouped']
df_train_exportado_words_onehot['state_info_grouped'] = df_train_exportado['state_info_grouped']

In [ ]:
# dfc = pd.get_dummies(dfc, columns=['state_info_grouped'], prefix='state_info_grouped', dtype=int)
df_train_exportado_words_onehot = pd.get_dummies(df_train_exportado_words_onehot, columns=['state_info_grouped'], prefix='state_info_grouped', dtype=int)
df_test_exportado_words_onehot = pd.get_dummies(df_test_exportado_words_onehot, columns=['state_info_grouped'], prefix='state_info_grouped', dtype=int)

## 5.4 party_affilation

In [ ]:
df_test_exportado_words_onehot['party_group'] = df_test_exportado['party_group']
df_train_exportado_words_onehot['party_group'] = df_train_exportado['party_group']

In [ ]:
# dfc = pd.get_dummies(dfc, columns=['party_group'], prefix='party_group', dtype=int)
df_train_exportado_words_onehot = pd.get_dummies(df_train_exportado_words_onehot, columns=['party_group'], prefix='party_group', dtype=int)
df_test_exportado_words_onehot = pd.get_dummies(df_test_exportado_words_onehot, columns=['party_group'], prefix='party_group', dtype=int)

# 6. Exportar CSV

In [ ]:
# Comparar las columnas de dfc y dfa
columns_train = set(df_train_exportado_words_onehot.columns)
columns_test = set(df_test_exportado_words_onehot.columns)

# Encontrar las diferencias
differences = columns_train.symmetric_difference(columns_test)

# Verificar si la única diferencia es 'id' y 'label'
if differences == {'id', 'label'}:
    print("La única diferencia entre las columnas es 'id' y 'label'.")
else:
    print("Existen otras diferencias en las columnas:", differences)
    print("Columnas en train pero no en test:", columns_train - columns_test)
    print("Columnas en test pero no en train:", columns_test - columns_train)

In [ ]:
# Exportar los datasets combinados a archivos CSV
df_train_exportado_words_onehot.to_csv('./formated/train_exportado_words.csv', index=False)
df_test_exportado_words_onehot.to_csv('./formated/test_exportado_words.csv', index=False)

print("Datasets combinados y exportados correctamente.")

# 7. Referencias

* [pandas documentation — pandas 2.2.3 documentation. (s. f.).](https://pandas.pydata.org/docs/)  